# Lab 1: Computational neurons, synapses, and motifs

In [ ]:
#@title Run the following to initialize lab environment.
import matplotlib.pyplot as plt         # import matplotlib
from matplotlib.widgets import Slider
import numpy as np                      # import numpy
import ipywidgets as widgets            # interactive display

# Colab setting for widget
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except ImportError:
    pass

# modeling library
from stg_net.neuron import LIF
from stg_net.input import Poisson_generator, Gaussian_generator, Current_injector
from stg_net.conn import Simulator
from stg_net.helper import plot_volt_trace

# setting for figures
fig_w, fig_h = 8, 6
my_fontsize = 18
my_params = {'axes.labelsize': my_fontsize,
          'axes.titlesize': my_fontsize,
          'figure.figsize': (fig_w, fig_h),
          'font.size': my_fontsize,
          'legend.fontsize': my_fontsize-4,
          'lines.markersize': 8.,
          'lines.linewidth': 2.,
          'xtick.labelsize': my_fontsize-2,
          'ytick.labelsize': my_fontsize-2}
plt.rcParams.update(my_params)
my_layout = widgets.Layout()

# Auto Reloading
%load_ext autoreload
%autoreload 2

# Widget interaction
%matplotlib widget

## Motifs

### Functional neural motifs (20min)

In this section, we move one step forward to connect those single neurons. What will happend with a pair of neurons? How about a triplet of neurons? Even more, a network of neurons?

We'll start with a simple neuron pair motif to introduce basic ideas of circuit level interactions.

<img src="https://github.com/michaelglh/STG/blob/master/figs/EI.png?raw=1" alt="EI motif" width="300"/>

In [ ]:
#@title Run the following to start E-I pair simulation { vertical-output: true }
# E-I motif
N = 2                  # number of neurons
wt, dl = 1, 5.

# updating parameters
def update_EI(Itype='Icur', neuron_E='tonic_neuron', neuron_I='tonic_neuron', J_ei=0., J_ie=0.):
    # simualtor
    h = Simulator(dt=dt)

    # network of neurons
    nrns = [LIF(sim=h) for _ in range(N)]

    nrns[0].update(neuron_params[neuron_E])
    nrns[1].update(neuron_params[neuron_I])

    # background noise
    if Itype == 'Icur':
        noises = [Current_injector(sim=h, rate=rt, start=int(T/dt*0.1), end=int(T/dt*0.9)) for _ in range(N)]
    elif Itype == 'Gaussian':
        noises = [Gaussian_generator(sim=h, mean=rt, std=rt) for _ in range(N)]
    elif Itype == 'Poisson':
        noises = [Poisson_generator(sim=h, rate=250) for _ in range(N)]
    else:
        print('Invalid input')
    for noise, nrn in zip(noises, nrns):
        nrn.connect(noise, {'ctype':'Static', 'weight':wt, 'delay':dl})

    # recurrent connections
    # tps = [['facilitate']*N]*N
    tps = [['Static']*N]*N
    con = np.array([[0., J_ei],
                    [J_ie, 0.]])
    dly = np.random.uniform(2., 5., (N,N))
    synspecs = [[{} for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            synspecs[i][j] = {'ctype':tps[i][j], 'weight':con[i,j], 'delay':dly[i,j]}
    h.connect(nrns, nrns, synspecs)

    # simulation
    h.run(T)

    # # extract data
    # bints = np.arange(0, T+1, 20.)
    # psths = [np.histogram(nrn.spikes['times'], bins=bints)[0] for nrn in nrns]

    # coincidence
    binwindow = int(5.0/dt)
    spike_trains = [nrn.states for nrn in nrns]
    bin_spikes = [np.convolve(strain, np.ones(binwindow), 'same') for strain in spike_trains]
    deltats = np.linspace(-10., 10., 21)
    coins = []
    for delay in deltats:
        index = int(delay/dt)
        if index > 0:
            coins.append(np.dot(bin_spikes[0][:-index], bin_spikes[1][index:]))
        elif index < 0:
            coins.append(np.dot(bin_spikes[1][:index], bin_spikes[0][-index:]))
        else:
            coins.append(np.dot(bin_spikes[0], bin_spikes[1]))
    coins = np.array(coins)/np.sqrt(np.dot(bin_spikes[0], bin_spikes[0]))/np.sqrt(np.dot(bin_spikes[1], bin_spikes[1]))

    # visualize
    plt.clf()
    cs = ['b', 'r']
    plt.subplot(2,N,1)
    plt.title('raster')
    for nrn, c, l in zip(nrns, cs, range(N)):
        plt.eventplot(nrn.spikes['times'], lineoffsets=2*l, colors=c)
    plt.xlabel('Time(ms)')
    plt.yticks([0, 2], ['E', 'I'])
    plt.xlim([0., T])

    plt.subplot(2,N,2)
    plt.title('spike correlation')
    plt.plot(deltats, coins)
    plt.xlabel(r'$\delta t$(ms)')
    plt.ylabel('spike corr')
    plt.ylim([0, 1])

    # voltage trace
    for id, c in zip(range(N), cs):
        plt.subplot(2,N,id+N+1)
        if id == 0:
            plt.title('voltage trace')
        plt_par = {'dt':dt, 'range_t':np.arange(0., T, dt), 'V_th':nrns[id].V_th}
        plot_volt_trace(plt_par, nrns[id].v, np.array(nrns[id].spikes['times']), c=c)
        plt.xlim([0., T])

    plt.tight_layout()

try:
  plt.figure(fig_EI)
  plt.clf()
except:
  print("Init figure")

fig_EI, axes = plt.subplots(2,N,figsize=(5*N,8))
widgets.interact(update_EI, Itype=['Icur', 'Gaussian', 'Poisson'], 
                 neuron_E=neuron_params.keys(), neuron_I=neuron_params.keys(), 
                 J_ie=(0.0, 50.0, 0.1), J_ei=(-50., 0., 0.1));

## Describe
- How are the excitatory and inhibitory synapse (connection) influence the firing rate of other neuron?
- How does the spike correlation change and why? Does input type change the shape of correlation? 
- What pattern do you observe at the motif level? How about changing the neuron types?
- If we drive the neurons with Poisson type inputs, they spike in an irregular manner. But can you find the connection weight which will make them spike in a periodic manner?

### For a pair of neurons, the influence of connections is straight-forward. *How will the situation change if adding one more neuron?* In the lab, we introduce one arbitrary triplet motif with only inhibitory connections.

<img src="https://github.com/michaelglh/STG/blob/master/figs/EII.png?raw=1" alt="EII motif" width="300"/>

<!-- Anatomical Divegent and Convergent

Functional Inhibitory;Disinhibitory -->

In [ ]:
#@title Run the following start E-I-I triplet simulation { vertical-output: true }
# Triplet
N = 3                  # number of neurons
wt, dl = 1., 5.
rt = 65.0

# updating parameters
def update_EII(Itype='Icur', neuron_E='tonic_neuron', neuron_I1='tonic_neuron', neuron_I2='tonic_neuron', J_E1=0., J_E2=0., J_21=0.):
    # simualtor
    h = Simulator(dt=dt)

    # network of neurons
    nrns = [LIF(sim=h) for _ in range(N)]
    nrns[0].update(neuron_params[neuron_E])
    nrns[1].update(neuron_params[neuron_I1])
    nrns[2].update(neuron_params[neuron_I2])

    # background noise
    if Itype == 'Icur':
        noises = [Current_injector(sim=h, rate=rt, start=int(T/dt*0.1), end=int(T/dt*0.9)) for _ in range(N)]
    elif Itype == 'Gaussian':
        noises = [Gaussian_generator(sim=h, mean=rt, std=rt) for _ in range(N)]
    elif Itype == 'Poisson':
        noises = [Poisson_generator(sim=h, rate=250) for _ in range(N)]
    else:
        print('Invalid input')
    for noise, nrn in zip(noises, nrns):
        nrn.connect(noise, {'ctype':'Static', 'weight':wt, 'delay':dl})

    # recurrent connections
    # tps = [['facilitate']*N]*N
    tps = [['Static']*N]*N
    con = np.array([[0., J_E1, J_E2],
                    [0., 0., 0.],
                    [0., J_21, 0.]])
    dly = np.random.uniform(2., 5., (N,N))
    synspecs = [[{} for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            synspecs[i][j] = {'ctype':tps[i][j], 'weight':con[i,j], 'delay':dly[i,j]}
    h.connect(nrns, nrns, synspecs)

    # simulation
    h.run(T)

    # coincidence
    binwindow = int(5.0/dt)
    spike_trains = [nrn.states for nrn in nrns]
    bin_spikes = [np.convolve(strain, np.ones(binwindow), 'same') for strain in spike_trains]
    deltats = np.linspace(-10., 10., 21)
    coins_all = []
    for src, tar in zip([0, 0], [1, 2]):
        coins = []
        for delay in deltats:
            index = int(delay/dt)
            if index > 0:
                coins.append(np.dot(bin_spikes[src][:-index], bin_spikes[tar][index:]))
            elif index < 0:
                coins.append(np.dot(bin_spikes[tar][:index], bin_spikes[src][-index:]))
            else:
                coins.append(np.dot(bin_spikes[src], bin_spikes[tar]))
        coins = np.array(coins)/np.sqrt(np.dot(bin_spikes[src], bin_spikes[src]))/np.sqrt(np.dot(bin_spikes[tar], bin_spikes[tar]))
        coins_all.append(coins)

    # visualize
    plt.clf()
    cs = ['b', 'r', 'g']  
    plt.subplot(2,N,1)
    plt.title('raster')
    for nrn, c, l in zip(nrns, cs, range(N)):
        plt.eventplot(nrn.spikes['times'], lineoffsets=2*l, colors=c)
    plt.xlabel('Time(ms)')
    plt.yticks(np.array(range(N))*2, ['E', r'$I_1$', r'$I_2$'])
    plt.xlim([0., T])

    # rate plots of population
    plt.subplot(2,N,2)
    plt.title('spike correlation')
    plt.plot(deltats, coins_all[0], 'r', label=r'$E-I_1$')
    plt.xlabel(r'$\delta t$(ms)')
    plt.ylabel('spike corr')
    plt.ylim([0,1])
    plt.legend()

    plt.subplot(2,N,3)
    plt.plot(deltats, coins_all[1], 'g', label=r'$E-I_2$')
    plt.xlabel(r'$\delta t$(ms)')
    plt.ylabel('spike corr')
    plt.ylim([0,1])
    plt.legend()

    # voltage trace
    for id, c in zip(range(N), cs):
        plt.subplot(2,N,id+N+1)
        if id == 0:
            plt.title('voltage trace')
        plt_par = {'dt':dt, 'range_t':np.arange(0., T, dt), 'V_th':nrns[id].V_th}
        plot_volt_trace(plt_par, nrns[id].v, np.array(nrns[id].spikes['times']), c=c)
        plt.xlim([0., T])
        
    plt.tight_layout()
      
try:
  plt.figure(fig_EII)
  plt.clf()
except:
  print("Init figure")

fig_EII, axes = plt.subplots(2, N,figsize=(5*N,8))
widgets.interact(update_EII, Itype=['Icur', 'Gaussian', 'Poisson'], 
                 neuron_E=neuron_params.keys(), neuron_I1=neuron_params.keys(), neuron_I2=neuron_params.keys(), 
                 J_E1=(-50., 0., 0.1), J_E2=(-50., 0., 0.1), J_21=(-50., 0., 0.1));

## Describe
- What are the effects of each inhibitory connection?
- What circuit motifs (anatomical and functional) co-exist in this 3-neuron network?
- Does inhibitory connectivity always reduce the firing rate of excitatory neurons?

## Think

- Does the input type influence the role of each connection? 
- What is dominant here, network connectivity or the neuron type? In other words which of your observation are invariant to change in neuron type and which are dependent on neuron type.